(case_studies)=
# Case studies

Description of the test cases
* *Motivate why they were selected as the candidates*
* *Where the data come from, measurement accuracy/uncertainty*
* *Describe the differences between the test cases to show how generally applicable the methodology is*
* *Success criteria: make a statement about which deviations or accuracy in the prediction are accepted as ''successful'' and why*


Two case study ship model test results, i.e., the wPCC as in {numref}`wpcc_mdl` and the well-known KVLCC2, are used to validate the proposed PIT method to obtain the hydrodynamic derivatives of the VMMs to describe a ship’s manoeuverability. The general aim with developing a VMM with PIT is to be able to make predictions for unseen data, outside the known data from model tests or full scale operation. The aim with both test cases is therefore to predict turning circle manouvres with VMMs that are trained on model tests data where turning circle manoeuvres are excluded. The training data contains drift angles and yaw rates that are much smaller than what is encounterd during a turning circle manoeuvre.
The main dimensions of those two case study ship models are listed in {numref}`Main_dimensions_of_test_case_ship_models`. The wPCC test model is a wind powered car carrier tested at SSPA. This twin-screw ship with large rudders has good course stabilility and is expected to have almost linear and symmetric hydrodynamic manoeuvring forces. The test setup and measurement procedures at SSPA is also well known to the authors which all in all makes it ideal as a first test case of the PIT.
The KVLCC2 test model and model test data from HSVA and MARIN was made available from SIMMAN2008 conference {cite:p}`stern_experience_2011`. This single screw ship is more course unstable than the wPCC test case and is therefore expected to have more nonlinear hydrodynamic manoeuvring forces which are also unsymetrical due to the single propeller. This makes it good as the seconds test case with PIT on a more nonlinear and unsymetrical model.

```{image} 
:alt: wpcc_mdl
:class: bg-primary mb-1
:width: 200px
:align: center
```

```{figure} figures/wpcc_mdl.png
---
height: 150px
name: wpcc_mdl
---
wPCC tested at SSPA. Copyright 2020 by SSPA Sweden AB. 
```

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
from jb_helpers import df_to_myst, parameter_to_latex
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force

In [ ]:
model_test_ids.keys()

In [ ]:
ships = ['wpcc','kvlcc2_hsva']

ship_datas = {}
initial_parameters = {}

for ship in ships:
       
    ship_datas[ship] = catalog.load(f"{ship}.ship_data")
    initial_parameters[ship] = catalog.load(f"{ship}.initial_parameters")
    

ship_datas = pd.DataFrame(ship_datas)
ship_datas.dropna(how='any', inplace=True)

initial_parameters = pd.DataFrame(initial_parameters)
initial_parameters.drop(index='Xthrust', inplace=True)
mask = (initial_parameters!=0).all(axis=1)
initial_parameters = initial_parameters.loc[mask].copy()

In [ ]:
ship_data_table = ship_datas.copy()
ship_data_table.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'}, inplace=True)
ship_data_table = ship_data_table.transpose()
ship_data_table[r'$k_{zz}$'] = np.sqrt(ship_data_table['I_z']/ship_data_table['m'])/ship_data_table['L']
ship_data_table.drop(columns=['rho','I_z','CB'], inplace=True)
ship_data_table.rename(
    columns={
        'T':'$T$ $[m]$',
        'L':'$L$ $[m]$',
        'B':'$B$ $[m]$',
        'x_G': '$L_{CG}$ $[m]$',
        'm': '$m$ $[kg]$',
        'I_z':'$I_{zz}$ $[kg m2]$',
        'volume':r'$\nabla$ $[m^3]$',
        'scale_factor':r'$\alpha$',
        'TWIN':'$N_p$',
        'w_p0':r'$w_{p0}$',
        'x_p':r'$x_{p}$ $[m]$',
        'x_r':r'$x_{r}$ $[m]$',
        'D':'$D$ $[m]$',
        'CB':r'$C_B$',
        
    },
    
    inplace=True)

ship_data_table['$N_p$']+=1
ship_data_table['$N_p$']=ship_data_table['$N_p$'].astype(int)
ship_data_table['$m$ $[kg]$']=ship_data_table['$m$ $[kg]$'].astype(int)

columns = ship_data_table.columns.sort_values()
ship_data_table=ship_data_table[columns].copy()
ship_data_table = ship_data_table.round(decimals=2)
#glue("ship_datas", ship_data_table)

print(df_to_myst(ship_data_table, title="Main dimensions of test case ship models"))
print('_____________')

```{list-table} Main dimensions of test case ship models
:header-rows: 1
:name: Main_dimensions_of_test_case_ship_models
* -  
  - $B$ $[m]$
  - $D$ $[m]$
  - $L$ $[m]$
  - $L_{CG}$ $[m]$
  - $N_p$
  - $T$ $[m]$
  - $\alpha$
  - $\nabla$ $[m^3]$
  - $k_{zz}$
  - $m$ $[kg]$
  - $w_{p0}$
  - $x_{p}$ $[m]$
  - $x_{r}$ $[m]$
* - WPCC
  - 0.95
  - 0.12
  - 5.01
  - 0.0
  - 2
  - 0.21
  - 41.2
  - 0.44
  - 0.25
  - 441
  - 0.15
  - -2.42
  - -2.42
* - KVLCC2 (HSVA)
  - 1.27
  - 0.2
  - 7.0
  - 0.24
  - 1
  - 0.46
  - 45.7
  - 3.27
  - 0.25
  - 3272
  - 0.4
  - -3.39
  - -3.5

```

### Initial conditions and assumptions

In [ ]:
s = ""
mask = df_parameters['brix'].notnull()
df_brix = df_parameters.loc[mask].copy()
df_brix.sort_index(inplace=True)
for index, row in df_brix.iterrows():
    
    eq = sp.Eq(row['symbol'],row['brix'])
    name = f"eq_{index}"
    glue(name,eq)
    
    s+=f"""
```{{glue:math}} {name}
:label: "{name}"
```
    """
print(s)   

The PIT requires an initial guessed linear model. Hydrodynamic derivatives for these inital models for the two test cases are shown in table {numref}`intial_guess`. These are calculated with semi empirical formulas ([eq](eq_Nr)- [eq](eq_Yvdot)) taken from {cite:p}`brix_manoeuvring_1993`. 


```{glue:math} eq_Nr
:label: "eq_Nr"
```
    
```{glue:math} eq_Nrdot
:label: "eq_Nrdot"
```
    
```{glue:math} eq_Nv
:label: "eq_Nv"
```
    
```{glue:math} eq_Nvdot
:label: "eq_Nvdot"
```
    
```{glue:math} eq_Xudot
:label: "eq_Xudot"
```
    
```{glue:math} eq_Yr
:label: "eq_Yr"
```
    
```{glue:math} eq_Yrdot
:label: "eq_Yrdot"
```
    
```{glue:math} eq_Yv
:label: "eq_Yv"
```
    
```{glue:math} eq_Yvdot
:label: "eq_Yvdot"
```
    


In [ ]:
initial_parameter_table = initial_parameters.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'})
initial_parameter_table = initial_parameter_table.transpose()
initial_parameter_table.columns = [parameter_to_latex(x) for x in initial_parameter_table.columns]

#glue("initial_parameters", 1000*initial_parameter_table)

print(df_to_myst((1000*initial_parameter_table).round(decimals=3), 
                 title="Initial guessed derivatives in linear models (times 1000)", 
                 name="intial_guess"))


```{list-table} Initial guessed derivatives in linear models (times 1000)
:header-rows: 1
:name: intial_guess
* -  
  - $ N_{\delta} $
  - $ N_{r} $
  - $ N_{\dot{r}} $
  - $ N_{v} $
  - $ N_{\dot{v}} $
  - $ X_{\dot{u}} $
  - $ Y_{\delta} $
  - $ Y_{r} $
  - $ Y_{\dot{r}} $
  - $ Y_{v} $
  - $ Y_{\dot{v}} $
* - WPCC
  - -1.5
  - -1.719
  - -0.299
  - -3.184
  - -0.128
  - 0.179
  - 3.0
  - 2.402
  - -0.303
  - -9.713
  - -6.109
* - KVLCC2 (HSVA)
  - -1.5
  - -3.415
  - -0.822
  - -8.707
  - -1.166
  - 1.05
  - 3.0
  - 4.305
  - -1.271
  - -25.266
  - -15.846

```

In [ ]:
def break_plot(df, **kwargs):
    
    df_ = df.copy()
    mask = np.concatenate([[False],np.diff(df_.index)>1])
    df_.loc[mask] = np.NaN
    df_.plot(**kwargs)


In [ ]:
def rotate(df):
    
    dpsi = 2*np.pi*np.random.random()
    
    df['psi']+=dpsi
    x0 = df['x0'].copy()
    y0 = df['y0'].copy()
    
    df['x0'] = np.cos(dpsi)*x0 - np.sin(dpsi)*y0
    df['y0'] = np.sin(dpsi)*x0 + np.cos(dpsi)*y0
        
    
    return df
    
    

### wPCC test scenarios

In [ ]:
ship="wpcc"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

#regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
#regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
#regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')


data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','id','x0','y0','psi']
data_with_force = predict_force(data=data[columns], added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
ids = list(data_with_force['id'].unique())
ids_train = ids.copy()
id_test = 22774
ids_train.remove(id_test)
mask = data_with_force['id'].isin(ids_train)
data_train = data_with_force.loc[mask].copy()
assert not id_test in data_train['id'].unique()

In [ ]:
ids_test = [
    22771,
    22772,
    22773,
]

mask = data_train['id'].isin(ids_test)
data_validation = data_train.loc[mask].copy()
data_sub_train = data_train.loc[~mask].copy()
mask = data['id']==id_test
data_testing = data_with_force.loc[mask]

A turning circle manoeuvre should be predicted for the wPCC test case based on a series of model tests including ZigZag10/10, 20/20 to port and startboard as well as self propulsion and yaw rate tests. The turning circle test contain much larger drift angles, rudder angle and yaw rates compared to the model tests used for training, so that the VMM:s prediction ability outside the traning data is tested.
This test case focuses on the prediction of forces and moments from the ship hull and rudders. The propeller force is therefore not part of the prediction model and is instead taken from the model test measurements.
The model test data that is used for training is split into a training and validation dataset. The training dataset contains self propulsion, yaw rate tests and zigzag10/10 tests to startboard and port. The validation dataset constists of three zigzag20/20 tests, so that the validation set contains larger drift angles, rudder angles and yaw rates than the training set in a similiar way as for the real prediction case. The training and validation datasets as well as the turning circle test case are shown in the. The training and validation datasets as well as the test set are shown in the {numref}`fig_traintest`.

```{glue:figure} fig_traintest
:name: "fig_traintest"
:figwidth: 1000px
wPCC training, validation and testing datasets.
```

In [ ]:
np.random.seed(3)
fig,ax=plt.subplots()

dataframes = {id:rotate(df_) for id, df_ in data_sub_train.groupby(by='id')}
styles = {id:{'style':'b-','label':'_nolegend_'} for id, df_ in data_sub_train.groupby(by='id')}

dataframes.update({id:rotate(df_) for id, df_ in data_validation.groupby(by='id')})
styles.update({id:{'style':'k--','label':'_nolegend_'} for id, df_ in data_validation.groupby(by='id')})

dataframes.update({id:rotate(df_) for id, df_ in data_testing.groupby(by='id')})
styles.update({id:{'style':'r-.','label':'_nolegend_'} for id, df_ in data_testing.groupby(by='id')})

track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=2, ax=ax);

ax.plot([],'b-',label='Training')
ax.plot([],'g-',label='Validation')
ax.plot([],'r-',label='Testing')


ax.legend();
glue('fig_traintest',fig, display=False)

### KVLCC2 test scenarios

In [ ]:
ship="kvlcc2_hsva"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

#regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
#regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
#regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')


data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")

added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
columns = ['u','v','r','u1d','v1d','r1d','delta','thrust','id','x0','y0','psi','rev']
data_with_force = predict_force(data=data[columns], added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)

In [ ]:
ids = list(data_with_force['id'].unique())
ids_train = ids.copy()
# id_test = 22774  (MARIN test as test)
#ids_train.remove(id_test)
mask = data_with_force['id'].isin(ids_train)
data_train = data_with_force.loc[mask].copy()
#assert not id_test in data_train['id'].unique()


In [ ]:
ids_test = [
    'HSVA_CPMC_KVLCC2_Z_35_05',
]

mask = data_train['id'].isin(ids_test)
data_validation = data_train.loc[mask].copy()
data_sub_train = data_train.loc[~mask].copy()

In [ ]:
ids2 = ["kvlcc2.updated.MARIN_FREE_KVLCC2_tc_35_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_tc_-35_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_10_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_-10_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_20_m",
        "kvlcc2.updated.MARIN_FREE_KVLCC2_zz_-20_m",
        
       ]

data_testing = pd.DataFrame()
for id in ids2: 
    df_test = catalog.load(f"{id}.data_ek_smooth")
    df_test['id'] = id
    data_testing = data_testing.append(df_test)
    

A turning circle manoeuvre should again be predicted, but now for the KVLCC2 case based on a series of model tests carried out at HSVA for the SIMMAN2008 conference {cite:p}`stern_experience_2011`. The turning circle test contain much larger drift angles, rudder angle and yaw rates compared to the model tests used for training, so that
the VMM:s prediction ability outside the traning data is tested.
The propeller is part of the VMM for this test case, instead of only hull and rudders as in the wPCC test case so that the full ship can be simulated without any additional input.
The model test data that is used for training is split into a training and validation dataset. The training
dataset contains various zigzag tests to startboard and port. The validation dataset constists of a ZigZag35/5
test, so that the validation set contains larger drift angles, rudder angles and yaw rates than the training set
in a similiar way as for the real prediction case.
The test set is taken from turning cirlce model tests caried out at MARIN for the SIMMAN2008 conference
{cite:p}`stern_experience_2011`. The training and validation datasets as well as the test set are shown in the {numref}`fig_kvlcc2_traintest`.

```{glue:figure} fig_kvlcc2_traintest
:name: "fig_kvlcc2_traintest"
:figwidth: 1000px
KVLCC2 training, validation and testing datasets.
```

In [ ]:
np.random.seed(1)
fig,ax=plt.subplots()

dataframes = {id:rotate(df_) for id, df_ in data_sub_train.groupby(by='id')}
styles = {id:{'style':'b-','label':'_nolegend_'} for id, df_ in data_sub_train.groupby(by='id')}

dataframes.update({id:rotate(df_) for id, df_ in data_validation.groupby(by='id')})
styles.update({id:{'style':'k--','label':'_nolegend_'} for id, df_ in data_validation.groupby(by='id')})

dataframes.update({id:rotate(df_) for id, df_ in data_testing.groupby(by='id') if 'tc' in id})
styles.update({id:{'style':'r-.','label':'_nolegend_'} for id, df_ in data_testing.groupby(by='id')})

track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles, N=2, ax=ax);

ax.plot([],'b-',label='Training (HSVA)')
ax.plot([],'g-',label='Validation (HSVA)')
ax.plot([],'r-',label='Testing (MARIN)')


ax.legend();
glue('fig_kvlcc2_traintest',fig, display=False)